# Importing Libraries

In [0]:
import re
import torch
import warnings
import numpy as np
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader

In [0]:
warnings.filterwarnings('ignore')

# Importing  Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd drive/My\ Drive/dataset

/content/drive/My Drive/dataset


In [6]:
!ls

cifar-10-python  processed_tag	  test.pickle
output.csv	 shakespeare.txt  train.pickle


In [0]:
with open('shakespeare.txt', 'r') as f:
    text = f.read()

In [8]:
print(text[:100])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose mi


# Pre processing text

In [0]:
class DataPreprocessing:
  
  def __init__(self,text):
    self.text = text
    
  def display_text(self):
    print(self.text)
    
  def  remove_numerals(self):
    return re.sub(r'[0-9]+','', self.text)
  
  def lower_text(self):
    return "".join(list(map(lambda x : x.lower(), self.text)))
  
  def remove_new_line(self):
    return re.sub("[\n]+"," ",self.text)
  
  def remove_multiple_spaces(self):
    return re.sub(' +', ' ',self.text)[1:]

In [0]:
processed_text = DataPreprocessing(text)

In [0]:
setattr(processed_text,'text',processed_text.remove_numerals())

In [0]:
setattr(processed_text,'text',processed_text.lower_text())

In [0]:
setattr(processed_text,'text',processed_text.remove_new_line())

In [0]:
setattr(processed_text, 'text', processed_text.remove_multiple_spaces())

In [15]:
len(getattr(processed_text, 'text'))

4925602

# Vectorization and One hot encoding

In [0]:
total_chars = tuple(set(getattr(processed_text, 'text')))

In [0]:
int2char = dict(enumerate(total_chars))

In [0]:
char2int = {ch:idx for idx,ch in int2char.items()}

In [37]:
len(processed_text.text)

4925602

In [0]:
encoded_text = np.array([char2int[char] for char in getattr(processed_text,'text')])

In [0]:
def onehot_encoding(encoded_val): 
  onehot_encoder = OneHotEncoder(sparse=False)
  onehot_encoded = onehot_encoder.fit_transform(encoded_val.reshape(len(encoded_val),1))
  
  return onehot_encoded

In [0]:
onehot_encoded = onehot_encoding(encoded_text)

In [0]:
CHAR_LEN = 25

In [0]:
train_dataset =[]
for idx in range(len(getattr(processed_text,'text'))-CHAR_LEN):
  train_dataset.append((onehot_encoded[idx:idx+CHAR_LEN].flatten(),onehot_encoded[idx+CHAR_LEN]))

In [1]:
loader = DataLoader(train_dataset, batch_size=10)

NameError: ignored

In [322]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class LanguageModel(nn.Module):
  
  def __init__(self,text,n_hidden=256,drop_prob=0.5,lr=0.001):
    super().__init__()
    self.text = text
    self.drop_prob = drop_prob
    self.n_hidden = n_hidden
    self.lr = lr
    
    
    

In [0]:
model = LanguageModel(getattr(processed_text,'text'))

In [367]:
model.onehot_encoder()

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])